# Library import

### download di YOLOv5 dalla repository github

In [ ]:
pip install -U ultralytics
pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

### import librerie

In [ ]:
from PIL import Image

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm

## Path

# Dataloader

In [ ]:
# la rete richiede che vengano date in input immagini in forma tensoriale -> per adattare le immagini vengono fornite delle funzioni apposite


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, txt_file, img_dir, utils):

        with open(txt_file, 'r') as f: #salva le region proposals in un file txt con direttamente le info della cartella
            self.image_paths = [line.strip() for line in f.readlines()] #memorizzo i path delle immagini in una lista
            
        self.inputs = [utils.prepare_input(image) for image in self.image_paths] # images = lista di path relativi alle immagini per il training
        self.tensor = utils.prepare_tensor(inputs)

    def __len__(self): # ritorna il numero di elementi in self.image_paths -> chi è?
        return len(self.image_paths)

    def __getitem__(self, index): # FINIRE DI COMPLETARE
        img_name = os.path.basename(self.image_paths[index]) #prendo il path dell'immagine da self.image_paths in base all'indice fornito
        img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', '')) #ricavo l'id dell'immagine -> non l'ho già fatto nell'init?
        
        if img_id not in self.image_info: #vedo dal dizionario self.image_info se l'immagine è contenuta nel file COCO 
            raise ValueError(f"Immagine {img_name} non trovata nel file COCO")
    
        img_path = os.path.join(self.img_dir, img_name) #prendo il path completo dell'immagine unendo il path della cartella con il nome.jpg dell'immagine
        if not os.path.exists(img_path): #se il path non esiste allora lo segnalo
            raise ValueError(f"Immagine non trovata nel percorso: {img_path}")
        
        image = Image.open(img_path).convert('RGB') #apro l'immagine e ne ricavo le dimensioni
        #original_width, original_height = image.size
        
        if self.aug: #se la variabile self.aug è alta allora applico la self.aug_transform altrimenti la self.base_transform _> HA SENSO? LE DUE FUNZIONI SONO = !!
            image_tensor = self.aug_transform(image)
        else:
            image_tensor = self.base_transform(image)
        
        #POTREBBE ESSERCI UN PROBLEMA NELLA CORRISPONDENZA TRA LABLES E REGION PROPOSALS -> perchè in proposals_tensor non ci sono tutte le region proposals perchè alcune vengono scartate
        # -> se la corrispondenza è 1 a 1 allora potrebbe convenire dare a _generate_region_proposals(image) sia le labes che l'immagine? -> non è certo perchè 
        # in lables c'è una lista di lable ma non viene specificato dove sono localizzate
        
        # restituisce un dizionario
        return {
            "regions": processed_proposals  # region proposals elaborate, una lista di tensori che rappresentano regioni candidate per il rilevamento
        } 

# Network

In [ ]:
# PER IL MODELLO YOLO
class YoloModel(nn.Module):
    def __init__(self, num_classes):
        super(YoloModel, self).__init__()

        ## Model -> per info https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Detection/SSD
        self.yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', classes = 12,  autoshape = False, pretrained=True)

    def forward(self, images):

        # Calcola le previsioni con il modello SSD
        predictions = self.yolo_model(images)  # Output grezzo del modello SSD
        
        return predictions

In [ ]:
class SSDModel(nn.Module):
    def __init__(self, num_classes):
        super(SSDModel, self).__init__()

        ## Model -> per info https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Detection/SSD
        self.ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd') # modello pre-addestrato su dataset COCO
        self.utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

    def forward(self, images):

        # Calcola le previsioni con il modello SSD
        predictions = self.ssd_model(images)  # Output grezzo del modello SSD
        
        return predictions

In [ ]:
class Trainer:
    def __init__(self, model, train_loader, val_loader, criterion, optimizer=None, device='cuda'):
        """
        Inizializza la classe Trainer.
        
        Args:
            model: Il modello SSD.
            train_loader: DataLoader per il training set.
            val_loader: DataLoader per il validation set.
            criterion: Funzione di perdita.
            optimizer: Ottimizzatore (opzionale, di default Adam).
            device: Dispositivo per il calcolo ('cuda' o 'cpu').
        """
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer if optimizer else Adam(model.parameters(), lr=1e-4)
        self.device = device
    
    def train_one_epoch(self, epoch):
        """
        Esegue un'epoca di addestramento.
        """
        self.model.train()
        running_loss = 0.0
        pbar = tqdm(self.train_loader, desc=f"Training Epoch {epoch}")
        
        for images, targets in pbar:
            images, targets = images.to(self.device), targets.to(self.device)
            
            # Forward pass
            predictions = self.model(images)
            
            # Calcolo della perdita
            loss = self.criterion(predictions, targets)
            
            # Backward pass e ottimizzazione
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            running_loss += loss.item()
            pbar.set_postfix({"loss": running_loss / len(self.train_loader)})
        
        return running_loss / len(self.train_loader)
    
    def validate_one_epoch(self, epoch):
        """
        Esegue un'epoca di validazione.
        """
        self.model.eval()
        running_loss = 0.0
        pbar = tqdm(self.val_loader, desc=f"Validation Epoch {epoch}")
        
        with torch.no_grad():
            for images, targets in pbar:
                images, targets = images.to(self.device), targets.to(self.device)
                
                # Forward pass
                predictions = self.model(images)
                
                # Calcolo della perdita
                loss = self.criterion(predictions, targets)
                running_loss += loss.item()
                
                pbar.set_postfix({"val_loss": running_loss / len(self.val_loader)})
        
        return running_loss / len(self.val_loader)
    
    def fit(self, epochs):
        """
        Esegue l'addestramento e la validazione per un dato numero di epoche.
        """
        train_losses = []
        val_losses = []
        
        for epoch in range(1, epochs + 1):
            train_loss = self.train_one_epoch(epoch)
            val_loss = self.validate_one_epoch(epoch)
            
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            
            print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        return train_losses, val_losses

In [ ]:
ssd_model.to('cuda')

# Training

# Testing